# Projet Spark - Covid VS Societé 2020

## Problematique
### Quelle est l'impacte du covid sur les société en France sur l'année 2020 ?


In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import *

In [2]:
spark = SparkSession.builder.appName("covid_societe").getOrCreate()

In [3]:
df_test = spark.read.csv("../data/donnees_covid_societes/donnees-hospitalieres-classe-age-covid19-2020-12-13-19h03.csv",header=True,sep=";")
df_test.show()
df_test.orderBy('jour', ascending=False).show()


+---+--------+----------+----+---+---+---+
|reg|cl_age90|      jour|hosp|rea|rad| dc|
+---+--------+----------+----+---+---+---+
| 01|       0|2020-03-18|   0|  0|  0|  0|
| 01|      09|2020-03-18|   0|  0|  0|  0|
| 01|      19|2020-03-18|   0|  0|  0|  0|
| 01|      29|2020-03-18|   0|  0|  0|  0|
| 01|      39|2020-03-18|   0|  0|  0|  0|
| 01|      49|2020-03-18|   0|  0|  0|  0|
| 01|      59|2020-03-18|   0|  0|  0|  0|
| 01|      69|2020-03-18|   0|  0|  0|  0|
| 01|      79|2020-03-18|   0|  0|  0|  0|
| 01|      89|2020-03-18|   0|  0|  0|  0|
| 01|      90|2020-03-18|   0|  0|  0|  0|
| 02|       0|2020-03-18|  16|  5|  0|  1|
| 02|      09|2020-03-18|   0|  0|  0|  0|
| 02|      19|2020-03-18|   0|  0|  0|  0|
| 02|      29|2020-03-18|   2|  1|  0|  0|
| 02|      39|2020-03-18|   2|  0|  0|  0|
| 02|      49|2020-03-18|   2|  1|  0|  0|
| 02|      59|2020-03-18|   5|  0|  0|  0|
| 02|      69|2020-03-18|   4|  3|  0|  0|
| 02|      79|2020-03-18|   1|  0|  0|  0|
+---+------